In [ ]:
import keras
keras.__version__

# Regresión: Prediciendo Precios de Casas

Pasamos de estudiar dos ejemplos de clasificación a un ejemplo de regresión, intentando predecir un valor continuo en vez de una etiqueta discreta.


## El Dataset de Precios de Casas de Boston

Intentaremos predecir el precio medio de casas de un suburbio de Boston (de datos de los años 70). Para ello, usaremos alguna información asociada a algunas casas de dicho suburbio: tasa de crimen, impuestos locales, etc.

A diferencia de los datasets vistos en los ejemplos anteriores, éste es realmente pequeño, apenas 506 anotaciones, que están divididas en 404/102 en entrenamiento/test. Además, cada una de las características de los datos de entrada usa una escala diferente, por ejemlo, algunos valores son proporciones (en `[0,1]`), otros toman valores entre 1 y 12, otros entre 0 y 100, etc.

Podemos echar un vistazo a los datos (es posible que la primera vez que ejecutes estas instrucciones se descargue el dataset, a que se accede por medio de instrucciones de la librería Keras):

In [ ]:
from keras.datasets import boston_housing

(train_data, train_targets), (test_data, test_targets) =  boston_housing.load_data()

In [ ]:
train_data.shape

In [ ]:
test_data.shape


Como se puede observar, cada dato tiene 13 características, que esperamos permitan obtener una relación funcional para predecir su precio, como por ejemplo:

1. Ratio de crimen per cápita.
2. Proporción de área residencial en lotes de 25,000 pies cuadrado.
3. ...

El objetivo es el valor medio de las casas, en miles de dólares:

In [ ]:
train_targets


Los precios están, aproximadamente, entre \$10,000 y \$50,000 (precios de los años 70, claro).


### Preparando los datos

Sería problemático alimentar la red neuronal con los datos en bruto que proporciona el dataset, que hace uso de rangos tan diferentes. Si fuera así, la red debería aprender, además de la relación funcional, a adaptar automáticamente la heterogeneidad de los datos, lo que haría que el aprendizaje fuera más complicado. Por ello, una beuna práctica de prepprocesamiento consiste en hacer una normalización de cada característica: para cada una de ellas (una columna) se extraer la media de sus valores y se divide por su desviación estándar, de esta forma la nueva características estará centrada en 0 y con desviación estándar 1. R permite hacer esta operación fácilmente con la función `scale()`, como veremos:

In [ ]:
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std

test_data -= mean
test_data /= std


Observa que las medias y desviaciones se calculan solo en los datos de entrenamiento, aunque se aplican a ambos conjuntos.


### Construyendo la red

Debido a que hay pocas muestras para el entrenamiento, vamos a usar una red muy pequeña, con solo 2 capas ocultas, cada una de 64 unidades. En general, cuantos menos datos tengamos de entrenamiento, mayor será el sobreajuste, así que usar una red pequeña puede mitigar este efecto.

In [ ]:
from keras import models
from keras import layers

def build_model():
    # Because we will need to instantiate
    # the same model multiple times,
    # we use a function to construct it.
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu',
                           input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model


Observa que la red acaba en una capa con una sola unidad y sin activación (es lo que se llama una capa lineal). Esta configuración es habitual cuando se hace regresión escalar (de un solo valor), ya que las funciones de activación restringen el rango de la salida, por ejemplo, una `sigmoid` aprende a predecir valores en `[0,1]`, pero una activación lineal puede aprender cualquier valor.

Observa también que se usa `mse` como función de pérdida, muy usada en el caso de problemas de regresión. Para monitorizar el entrenamiento usamos `mae`, *Mean Absolute Error*, que es el valor absoluto de la diferencia entre las predicciones y los objetivos. Por ejemplo, un MAE de 0.5 en este problema significa que nuestras predicciones difieren en unos \$500 de media.

### Validando nuestra aproximación usando K-validación cruzada

Para evaluar la red mientras ajustamos los hiperparámetros (como el número de epochs a usar), podríamos dividir simplemente el conjunto de entrenamiento en entrenamiento y validación, tal y como hicimos en ejemplos anteriores. Sin embargo, debido a que hay muy pocas muestras, el conjunto de validación acabaría siendo muy pequeño (unas 100 muestras), por lo que las métricas de validación pueden depender excesivamente de qué muestras concretas han terminado en cada conjunto, mostrando, posiblemente, una gran varianza.

La mejor práctica en este tipo de situaciones es usar una K-validación cruzada, que consiste em dividir los datos disponibles en K particiones (normalmente, entre 4 y 5), después instanciar K modelos exactos, y entonces entrenar cada uno de los modelos usando K-1 de las particiones anteriores y evaluar sobre la restante. La métrica de evaluación será la media de las K métricas obtenidas sobre las validaciones de cada modelo.

Un código en R que nos permite realizar esta tarea podría ser:

In [ ]:
import numpy as np

k = 4
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []
for i in range(k):
    print('processing fold #', i)
    # Prepare the validation data: data from partition # k
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]

    # Prepare the training data: data from all other partitions
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
         train_data[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
         train_targets[(i + 1) * num_val_samples:]],
        axis=0)

    # Build the Keras model (already compiled)
    model = build_model()
    # Train the model (in silent mode, verbose=0)
    model.fit(partial_train_data, partial_train_targets,
              epochs=num_epochs, batch_size=1, verbose=0)
    # Evaluate the model on the validation data
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)

In [ ]:
all_scores

In [ ]:
np.mean(all_scores)


Como puedes observar, las posibles métricas de validación varían mucho (dependiendo de la ejecución, de 2.1 a 3.2), por lo que la media (2.55) es mucho más fiable que cada una de las otras valoraciones por separado... este es precisamente el valor de la K-vlidacion cruzada. Todavía estamos con un error significativo de unos $2,550 en precios que estan en el rango \$10,000 to \$50,000, así que intentemos entrenar a la red durante un poco más de tiempo: 200 epochs. Con el fin de guardar un registro de cómo de bien funciona el modelo en cada epoch, vamos a modificar el bucle de entrenamiento para almacenar el score de validación en cada epoch:


In [ ]:
from keras import backend as K

# Some memory clean-up
K.clear_session()

In [ ]:
num_epochs = 500
all_mae_histories = []
for i in range(k):
    print('processing fold #', i)
    # Prepare the validation data: data from partition # k
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]

    # Prepare the training data: data from all other partitions
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
         train_data[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
         train_targets[(i + 1) * num_val_samples:]],
        axis=0)

    # Build the Keras model (already compiled)
    model = build_model()
    # Train the model (in silent mode, verbose=0)
    history = model.fit(partial_train_data, partial_train_targets,
                        validation_data=(val_data, val_targets),
                        epochs=num_epochs, batch_size=1, verbose=0)
    mae_history = history.history['val_mean_absolute_error']
    all_mae_histories.append(mae_history)

Ahora podemos calcular la media de los valores MAE en cada epoch:


In [ ]:
average_mae_history = [
    np.mean([x[i] for x in all_mae_histories]) for i in range(num_epochs)]

Y representarlo:


In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(1, len(average_mae_history) + 1), average_mae_history)
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')
plt.show()


Puede ser un poco difícil extraer conocimiento de esta gráfica debido a los problemas de escala que presenta y a la alta varianza, así que podemos hacer lo siguiente:

* Omitir los primeros 10 puntos de los datos, que parecen mostrar una escala distinta al resto de la curva.
* Reemplazar cada punto con una media exponencial de los puntos anteriores, con el fin de obtener una curva más suave.

In [ ]:
def smooth_curve(points, factor=0.9):
  smoothed_points = []
  for point in points:
    if smoothed_points:
      previous = smoothed_points[-1]
      smoothed_points.append(previous * factor + point * (1 - factor))
    else:
      smoothed_points.append(point)
  return smoothed_points

smooth_mae_history = smooth_curve(average_mae_history[10:])

plt.plot(range(1, len(smooth_mae_history) + 1), smooth_mae_history)
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')
plt.show()


De acuerdo con esta nueva gráfica, parece que el MAE de validación deja de mejorar tras 100-130 epochs, a partir de entonces empieza a haber sobreajuste.

Tras haber ajustado otros parámetros del modelo (por ejemplo, el tamaño de las capas ocultas), podemos entrenar una versión final del modelo sobre todos los datos de entrenamiento y medir el rendimiento sobre los datos de test:

In [ ]:
# Get a fresh, compiled model.
model = build_model()
# Train it on the entirety of the data.
model.fit(train_data, train_targets,
          epochs=80, batch_size=16, verbose=0)
test_mse_score, test_mae_score = model.evaluate(test_data, test_targets)

In [ ]:
test_mae_score

Que sigue mostrando un error todavía muy alto, rondando los \$2,500.


## Conclusiones

* La regresión hace uso de funciones de pérdida distintas a la clasificación. La más común suele ser la *Mean Squared Error* (MSE).
* Además, las métricas de evaluación también suelen ser distintas, por ejemplo, *Mean Absolute Error* (MAE). El término "accuracy" aquí no tiene sentido.
* Cuando las características de los datos de entrada usan diferentes rangos, cada una de ellas ha de ser normalizada independientemente en la etapa de preprocesamiento.
* Cuando tenemos pocos datos, usar K-validación cruzada puede ser un buen método para evaluar el modelo de forma más fiable.
* Cuando hay pocosa datos, es preferible usar redes pequeñas con pocas capas (normalmente, una o dos) con el fin de evitar un sobreajuste exagerado.